In [ ]:
import datetime # импорт datetime

class Account: # создание класса аккаунт
    def __init__(self, account_holder: str, balance: float = 0):
        self.holder = account_holder
        self._balance = balance
        self.operations_history = []  # Список для хранения операций
        if balance < 0: # проверка стартового баланса 
            raise ValueError("Начальный баланс не может быть отрицательным.")
 
    def deposit(self, amount): # пополнение счета
        if amount <= 0:
            raise ValueError("Сумма пополнения должна быть положительной.")
        self._balance += amount
        self.addoperation('deposit', amount, 'success')

    def withdraw(self, amount):  #снятие средств
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной.")
        if self._balance >= amount:
            self._balance -= amount
            self.addoperation('withdraw', amount, 'success')
        else:
            self.addoperation('withdraw', amount, 'fail')

    def get_balance(self):
        return self._balance

    def get_history(self):
        return self.operations_history

    def addoperation(self, operationtype, amount, status):
        operation = {
            'type': operationtype,
            'amount': amount,
            'date_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # формат времени операции для журнала
            'balanceafter': self._balance,
            'status': status
        }
        self.operations_history.append(operation) #сохранение операций

class CreditAccount(Account): # создание класса CreditAccount
    def __init__(self, account_holder: str, balance: float = 0, credit_limit: float = 0):
        super().__init__(account_holder, balance)
        if credit_limit < 0:
            raise ValueError("Кредитный лимит не может быть отрицательным.")
        self.credit_limit = credit_limit

    def withdraw(self, amount: float): # создадим функцию снятия средств
        if amount <= 0:
            self._add_operation_with_credit_info('withdraw', amount, 'fail', False)
            return False

        available_funds = self._balance + self.credit_limit
        if available_funds < amount:
            self._add_operation_with_credit_info('withdraw', amount, 'fail', False)
            return False

        self._balance -= amount
        used_credit = self._balance < 0
        self._add_operation_with_credit_info('withdraw', amount, 'success', used_credit)
        return True

    def get_available_credit(self) -> float:
        return max(0, self.credit_limit + self._balance)

    def _add_operation_with_credit_info(self, operation_type: str, amount: float, status: str, used_credit: bool):
        operation = {
            'type': operation_type,
            'amount': amount,
            'date_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'balance_after': self._balance,
            'status': status,
            'used_credit': used_credit
        }
        self.operations_history.append(operation)
    def get_available_credit(self) -> float: # функция выдачи в пределах кредитного лимита
        return self.credit_limit + self._balance

#Создаем дебетовый счет       
deb = Account("Елена", 7000)
deb.deposit(3500)
deb.withdraw(1050)
deb.withdraw(7000) 
deb.withdraw(1000) 
deb.withdraw(2000) 
print("Текущий баланс:", deb.get_balance())
for row in deb.get_history():
    print(row)

#Создаем кредитный счет
print("Кредит")
cred = CreditAccount("Елена", balance=910, credit_limit=2800)
cred.withdraw(2450)   
cred.withdraw(14000)   
cred.withdraw(5000)
cred.deposit(560)    
cred.deposit(400)
print("Текущий баланс:", cred.get_balance())
print("Доступные кредитные средства:", cred.get_available_credit())
for row in cred.get_history():
    print(row)
